In [1]:
from pokemontcgsdk import Card, Set, RestClient
from PIL import Image
from io import BytesIO
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from Utils.cardParser import extractCore
import time
from requests.exceptions import ConnectionError, Timeout, HTTPError
import os
import numpy as np
from tqdm.notebook import tqdm

In [2]:
api_key = "api-key"
RestClient.configure(api_key)

In [ ]:
cards = Card.all()#fetch all cards from the sdk

In [3]:
#Parse the objects to a dataframe each card as a row
#df = pd.DataFrame(columns=["id", "name", "rarity", "superType", "img_path_small"])

#for card in tqdm(cards, total = len(cards), desc="Processing data"):
#    df.loc[len(df)] = extractCore(card)

df = pd.read_parquet("ALL_CARDS.parquet")

In [4]:
model = SentenceTransformer("clip-ViT-B-16")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
embeddings = []

In [10]:
df.rename(columns={"id":"id", "name":"name", "rarity":"rarity", "small_image":"img_path_small", "supertype":"supertype"}, inplace=True)
df.head(4)

,id,name,rarity,img_path_small,supertype
0,base1-1,Alakazam,Rare Holo,https://images.pokemontcg.io/base1/1.png,Pokémon
1,base1-2,Blastoise,Rare Holo,https://images.pokemontcg.io/base1/2.png,Pokémon
2,base1-3,Chansey,Rare Holo,https://images.pokemontcg.io/base1/3.png,Pokémon
3,base1-4,Charizard,Rare Holo,https://images.pokemontcg.io/base1/4.png,Pokémon


In [11]:
for index, c in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):

    img = requests.get(c["img_path_small"]).content
    img = Image.open(BytesIO(img))
    
    emb = model.encode(img, normalize_embeddings=True)
    embeddings.append(emb)

Processing Rows:   0%|          | 0/19653 [00:00<?, ?it/s]

c:\Users\lucas\anaconda3\envs\Gastly\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [12]:
df["embedding"] = embeddings

In [13]:
df.head(10)

,id,name,rarity,img_path_small,supertype,embedding
0,base1-1,Alakazam,Rare Holo,https://images.pokemontcg.io/base1/1.png,Pokémon,"[0.050831325, -0.03796756, 0.024911977, 0.0140..."
1,base1-2,Blastoise,Rare Holo,https://images.pokemontcg.io/base1/2.png,Pokémon,"[0.045375768, -0.053173628, 0.03134433, 0.0273..."
2,base1-3,Chansey,Rare Holo,https://images.pokemontcg.io/base1/3.png,Pokémon,"[0.10473484, -0.053638723, 0.021468604, 0.0194..."
3,base1-4,Charizard,Rare Holo,https://images.pokemontcg.io/base1/4.png,Pokémon,"[0.043872546, -0.052383587, 0.0032058612, 0.04..."
4,base1-5,Clefairy,Rare Holo,https://images.pokemontcg.io/base1/5.png,Pokémon,"[0.07277652, -0.037050337, 0.0074084904, 0.041..."
5,base1-6,Gyarados,Rare Holo,https://images.pokemontcg.io/base1/6.png,Pokémon,"[0.072361045, -0.087607674, 0.011755845, 0.012..."
6,base1-7,Hitmonchan,Rare Holo,https://images.pokemontcg.io/base1/7.png,Pokémon,"[0.05420909, -0.045752537, 0.01779943, 0.00174..."
7,base1-8,Machamp,Rare Holo,https://images.pokemontcg.io/base1/8.png,Pokémon,"[0.066932246, -0.049026825, 0.026209813, 0.020..."
8,base1-9,Magneton,Rare Holo,https://images.pokemontcg.io/base1/9.png,Pokémon,"[0.061491445, -0.06472015, 0.013530687, 0.0082..."
9,base1-10,Mewtwo,Rare Holo,https://images.pokemontcg.io/base1/10.png,Pokémon,"[0.048385024, -0.03590499, 0.035184816, 0.0423..."


In [14]:
df.to_parquet("FULL_MASTER_SET_b16.parquet")